<a href="https://colab.research.google.com/github/clu97123/CAIS_Winter_Project/blob/main/HSP_Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

code mostly taken from here: https://github.com/bentrevett/pytorch-seq2seq/blob/main/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
# Helper libraries
import pandas as pd
import random
from tqdm import tqdm
import numpy as np
import spacy
!pip install datasets
!pip install evaluate
import datasets
import evaluate
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [ ]:
seed = 52
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

aave_dir = '/content/drive/MyDrive/HSP_S2S/aave_samples_cleaned'
sae_dir = '/content/drive/MyDrive/HSP_S2S/sae_samples'
aave_set = datasets.load_dataset(aave_dir)
sae_set = datasets.load_dataset(sae_dir)
aave_set = aave_set.rename_column("text", "aave")
sae_set = sae_set.rename_column("text", "sae")
aave_set

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


DatasetDict({
    train: Dataset({
        features: ['aave'],
        num_rows: 2019
    })
})

In [ ]:
dialect_set = datasets.concatenate_datasets([sae_set['train'], aave_set['train']], axis=1)
dialect_set

Dataset({
    features: ['sae', 'aave'],
    num_rows: 2019
})

In [ ]:
train_testvalid = dialect_set.train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
dialect_set = datasets.DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']
})

In [ ]:
train_data, test_data, valid_data = (
    dialect_set['train'],
    dialect_set['test'],
    dialect_set['valid']
)
print(train_data[0])
print(test_data[0])
print(valid_data[0])

{'sae': "I've got two tickets to paradise. Woo woo woo woo.Woo woo woo", 'aave': 'Ive Got. . 2Tickets To PARADISE . . . , Wo Wo Wo Woo.. Wooo Wooo Wooo .'}
{'sae': "and how long ago was that? I'm laughing hysterically. I been since I could remember now...I could shed a tear I'mso happy for you. I'm laughing", 'aave': "and how long ago was that? Lmfaoo. I been since I could remember now. . I could shed a tear I'mso happy for u lol"}
{'sae': 'My mother disappoints me sometimes...why does my lifehave to be harder? gosh.', 'aave': 'My mom blows the fuck out of me at times... Fuck my lifehave to harder for? Damn'}


In [ ]:
en_nlp = spacy.load('en_core_web_sm')

In [ ]:
def tokenize_example(example, en_nlp, max_len, lower, sos_token, eos_token):
    sae_tokens = [token.text for token in en_nlp.tokenizer(example["sae"])][:max_len]
    aave_tokens = [token.text for token in en_nlp.tokenizer(example["aave"])][:max_len]
    if lower:
        sae_tokens = [token.lower() for token in sae_tokens]
        aave_tokens = [token.lower() for token in aave_tokens]
    sae_tokens = [sos_token] + sae_tokens + [eos_token]
    aave_tokens = [sos_token] + aave_tokens + [eos_token]
    return {"sae_tokens": sae_tokens, "aave_tokens": aave_tokens}

In [ ]:
max_len = 1000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp": en_nlp,
    "max_len": max_len,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token
}

train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/1615 [00:00<?, ? examples/s]

Map:   0%|          | 0/202 [00:00<?, ? examples/s]

Map:   0%|          | 0/202 [00:00<?, ? examples/s]

In [ ]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token
]

sae_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["sae_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

aave_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["aave_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

In [ ]:
len(sae_vocab), len(aave_vocab)

(2009, 2114)

In [ ]:
assert sae_vocab[unk_token] == aave_vocab[unk_token]
assert sae_vocab[pad_token] == aave_vocab[pad_token]

unk_index = sae_vocab[unk_token]
pad_index = sae_vocab[pad_token]

In [ ]:
sae_vocab.set_default_index(unk_index)
aave_vocab.set_default_index(unk_index)

In [ ]:
def numericalize_example(example, sae_vocab, aave_vocab):
    sae_ids = sae_vocab.lookup_indices(example['sae_tokens'])
    aave_ids = aave_vocab.lookup_indices(example['aave_tokens'])
    return {"sae_ids": sae_ids, "aave_ids": aave_ids}

In [ ]:
fn_kwargs = {"sae_vocab": sae_vocab, "aave_vocab": aave_vocab}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/1615 [00:00<?, ? examples/s]

Map:   0%|          | 0/202 [00:00<?, ? examples/s]

Map:   0%|          | 0/202 [00:00<?, ? examples/s]

In [ ]:
data_type = "torch"
format_columns = ["sae_ids", "aave_ids"]

train_data = train_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True
)

In [ ]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_sae_ids = [example['sae_ids'] for example in batch]
        batch_aave_ids = [example['aave_ids'] for example in batch]
        batch_sae_ids = nn.utils.rnn.pad_sequence(batch_sae_ids, padding_value=pad_index)
        batch_aave_ids = nn.utils.rnn.pad_sequence(batch_aave_ids, padding_value=pad_index)
        batch = {
            "sae_ids": batch_sae_ids,
            "aave_ids": batch_aave_ids
        }
        return batch

    return collate_fn


In [ ]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle
    )
    return data_loader

In [ ]:
batch_size = 32

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

In [ ]:
input_dim = len(sae_vocab)
output_dim = len(aave_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2009, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(2114, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=2114, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 9,496,386 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [ ]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["sae_ids"].to(device)
        trg = batch["aave_ids"].to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["sae_ids"].to(device)
            trg = batch["aave_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
# training time aka this is where my laptop blows up

n_epochs = 11
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "HSP_S2S_Proto.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

  9%|▉         | 1/11 [02:33<25:36, 153.64s/it]

	Train Loss:   5.873 | Train PPL: 355.271
	Valid Loss:   5.213 | Valid PPL: 183.578


 18%|█▊        | 2/11 [05:01<22:32, 150.29s/it]

	Train Loss:   5.560 | Train PPL: 259.807
	Valid Loss:   5.176 | Valid PPL: 177.039


 27%|██▋       | 3/11 [07:28<19:48, 148.60s/it]

...

	Train Loss:   5.252 | Train PPL: 190.935
	Valid Loss:   5.244 | Valid PPL: 189.394


 91%|█████████ | 10/11 [24:28<02:26, 146.05s/it]

	Train Loss:   5.201 | Train PPL: 181.504
	Valid Loss:   5.250 | Valid PPL: 190.505


100%|██████████| 11/11 [26:55<00:00, 146.90s/it]

	Train Loss:   5.168 | Train PPL: 175.651
	Valid Loss:   5.322 | Valid PPL: 204.817


In [ ]:
model.load_state_dict(torch.load("HSP_S2S_Proto.pt"))

test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

| Test Loss: 5.299 | Test PPL: 200.093 |


In [ ]:
def translate_sentence(
    sentence,
    model,
    en_nlp,
    sae_vocab,
    aave_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = [token.text for token in en_nlp.tokenizer(sentence)]
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = sae_vocab.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        hidden, cell = model.encoder(tensor)
        inputs = aave_vocab.lookup_indices([sos_token])
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == aave_vocab[eos_token]:
                break
        tokens = aave_vocab.lookup_tokens(inputs)
    return tokens

In [ ]:
sentence = test_data[0]["sae"]
expected_translation = test_data[0]["aave"]

sentence, expected_translation

("and how long ago was that? I'm laughing hysterically. I been since I could remember now...I could shed a tear I'mso happy for you. I'm laughing",
 "and how long ago was that? Lmfaoo. I been since I could remember now. . I could shed a tear I'mso happy for u lol")

In [ ]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    aave_vocab,
    sae_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

translation
# lol lmao even. idt this works

['<sos>',
 'i',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>',
 '<eos>']

In [ ]:
translations = [
    translate_sentence(
        example["sae"],
        model,
        en_nlp,
        aave_vocab,
        sae_vocab,
        lower,
        sos_token,
        eos_token,
        device,
    )
    for example in tqdm(test_data)
]
# gave up here


 18%|█▊        | 36/202 [00:05<00:26,  6.20it/s]

IndexError: index out of range in self